In [1]:
import numpy as np
import pandas as pd
import re
import pickle
from pprint import pprint


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json
data = json.loads('{"one" : "1", "two" : "2", "three" : "3"}')

from bson import json_util

from mongoengine import *

In [2]:
from flask import Flask,jsonify, request, send_from_directory, url_for
app = Flask(__name__)
import pickle
import numpy as np
from itertools import chain

In [3]:
conn = connect("local",host="mongodb://localhost:27017",alias='default')

if(0):
    disconnect('mongo://mongo:27017')

In [4]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))
if(0):
    pickle.dump(str_menu,open('half_menu_main.pkl','wb'))
restautant_id = 'BNGHSR0001'

final_list_json=[]

In [5]:
def chainer(string):
    return list(chain.from_iterable(string.str.split('/')))

def str_2(number):
    string = str(number)
    if(len(string)==1):
        return '0'+string
    else:
        return string

In [6]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        

In [92]:

class FoodItem(Document):
    pass
class Table(Document):
    pass

class FoodOptions(DynamicDocument):
    size_field = DictField()
    best_paired_with = ListField(FoodItem)

class FoodItem(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    price = StringField(required=True)
    foodoptions = ReferenceField(FoodOptions)
    
    def to_mymongo(self):
        data = self.to_mongo()
        if(self.foodoptions):
            data['foodoptions']=self.foodoptions.to_mongo()

        return data
    
    def to_json(self):
        data = self.to_mongo()
        if(self.foodoptions):
            data['foodoptions']=self.foodoptions.to_mongo()

        return json_util.dumps(data)

class SubCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    foodlist= ListField(ReferenceField(FoodItem),required=True)
    
    def to_mymongo(self):
        data = self.to_mongo()
        for key, fooditem in enumerate(self.foodlist):
            data['foodlist'][key]=self.foodlist[key].to_mymongo()

        return data

class MainCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    sub_category= ListField(ReferenceField(SubCategory),required=True)
    
    def to_mymongo(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.sub_category):
            data['sub_category'][key]=self.sub_category[key].to_mymongo()

        return data


class Restaurant(Document):
    name = StringField(required=True)
    restaurant_id = StringField(required=True)
    menu = ListField(ReferenceField(MainCategory),required=True)
    address = StringField()
    tables = ListField(ReferenceField(Table))
    servers = StringField()
    
    def to_json(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.menu):
            data['menu'][key]=self.menu[key].to_mymongo()

        return json_util.dumps(data)




class User(Document):
    dine_in_history = DynamicField()
    current_table = ReferenceField(Table)
    personal_cart = ListField(ReferenceField(Order))
    meta = {'allow_inheritance': True}
    
class TempUser(User):
    pass
class RegisteredUser(User):
    name = StringField(required=True)
    email_id = StringField(required=True)
    phone_no = StringField()
    tempuser_ob = ReferenceField(TempUser)
    
class TempUser(User):
    unique_id = StringField(required=True)
    reguser_ob = ReferenceField(RegisteredUser)
    
class Table(Document):
    name=StringField(required=True)
    seats=IntField(required=True)
    servers = StringField()
    users = ListField(ReferenceField(User))
    nofusers = IntField()
    orders = ListField(ReferenceField(Order))

In [9]:
FoodItem.drop_collection()
SubCategory.drop_collection()
FoodOptions.drop_collection()
MainCategory.drop_collection()
Restaurant.drop_collection()
Table.drop_collection()
TempUser.drop_collection()
RegisteredUser.drop_collection()

main_cat_list = []
for p,main_cat in enumerate(final_list_json):
    sub_cat_list = []
    for q,sub_cat in enumerate(main_cat['sub_category']):
        food_list = []
        for r,food in enumerate(sub_cat['foodlist']):
            food_list.append(FoodItem(name=food['name'],description = food['description'],price=food['price']).save())
        sub_cat_list.append(SubCategory(name=sub_cat['name'],description=sub_cat['description'],foodlist = food_list).save())
    main_cat_list.append(MainCategory(name=main_cat['name'],description = main_cat['description'],sub_category = sub_cat_list).save())

houseofcommons=Restaurant(name = 'House of Commons',restaurant_id='BNGHSR0001',menu = main_cat_list,address='')
houseofcommons.save()

FoodItem.objects.filter(name="TANDOORI MURGH (HALF/FULL)")[0].update(set__foodoptions=FoodOptions(size_field={'HALF':'250','FULL':'480'}).save())

1

In [10]:
table_list=[]
for n in range(1,11):
    table_list.append(Table(name='table'+str(n),seats = 6,nofusers=0).save().to_dbref())

Restaurant.objects[0].update(set__tables=table_list)

1

In [11]:
table_list

[DBRef('table', ObjectId('5e77a5c8bd1fb4e23bfd9f08')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f09')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0a')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0b')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0c')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0d')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0e')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0f')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f10')),
 DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f11'))]

In [12]:
akshay = TempUser(unique_id = "hfirnivnhhwocn34534no34n34r")

akshay.save()

RegisteredUser(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').save()

RegisteredUser.objects(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').update(set__tempuser_ob=akshay.to_dbref())

1

In [13]:
def user_scan(table_id,unique_id,email_id='dud'):
    scanned_table=Table.objects.get(id=table_id)
    
    if(email_id=='dud'):
        tempuser = TempUser.objects.filter(unique_id=unique_id)
        if(len(tempuser)>0):
            tempuser[0].update(set__current_table=scanned_table.to_dbref())
            scanned_table.update(push__users=tempuser[0].to_dbref())
        else:
            tempuser= TempUser(unique_id=unique_id,current_table=scanned_table.to_dbref()).save()
            scanned_table.update(push__users=tempuser.to_dbref())
        scanned_table.update(inc__nofusers=1)
    else:
        reg_user=RegisteredUser.objects.filter(email_id=email_id)[0]
        scanned_table.update(push__users=reg_user)
        reg_user.update(set__current_table=scanned_table.to_dbref())
        scanned_table.update(inc__nofusers=1)

In [14]:
user_scan(str(table_list[2].id),'hidnfvidfkvmsdmv')

In [15]:
user_scan(str(table_list[2].id),'hidnfvidvdff1kvmsadmv')

In [16]:
user_scan(str(table_list[2].id),'hidnfvidvdffaskvmsadmv')

In [17]:
user_scan(str(table_list[2].id),'hidnfvidvdffvf1kvmsadmv')

In [18]:
user_scan(str(table_list[2].id),'',email_id='akshay.dn5@gmail.com')

In [31]:
class FoodOptionsMod(EmbeddedDocument):
    size_field = DictField()

class FoodItemMod(EmbeddedDocument):
    name=StringField()
    description=StringField()
    price=StringField()
    instructions=StringField()
    foodoptions= EmbeddedDocumentField(FoodOptionsMod)
    
class Order(Document):
    placedby = ListField(ReferenceField(User))
    table = ReferenceField(Table)
    foodlist = MapField(EmbeddedDocumentField(FoodItemMod))
    customer_type = StringField()


In [20]:
FoodItem.objects.exclude("id")[45].to_json()

'{"name": "TANDOORI MURGH (HALF/FULL)", "description": "Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor", "price": "250", "foodoptions": {"_id": {"$oid": "5e77a5c7bd1fb4e23bfd9f07"}, "size_field": {"HALF": "250", "FULL": "480"}, "best_paired_with": []}}'

In [21]:
hell=FoodItemMod.from_json(FoodItem.objects.exclude("id")[0].to_json())

In [22]:
hell

<FoodItemMod: FoodItemMod object>

In [33]:
User.objects[0].to_dbref()

DBRef('user', ObjectId('5e77a5cbbd1fb4e23bfd9f13'))

In [91]:
FoodItem.objects[42].to_dbref()

DBRef('food_item', ObjectId('5e77a5c7bd1fb4e23bfd9ee6'))

In [53]:
input_order={'placedby' : ["5e77a5cbbd1fb4e23bfd9f13"],'table' : '5e77a5c8bd1fb4e23bfd9f08','foodlist' : ['5e77a5c7bd1fb4e23bfd9f01','5e77a5c7bd1fb4e23bfd9ef4','5e77a5c7bd1fb4e23bfd9edf'],'foodmod_list':{},'order_type':'user'}
input_order

{'placedby': ['5e77a5cbbd1fb4e23bfd9f13'],
 'table': '5e77a5c8bd1fb4e23bfd9f08',
 'foodlist': ['5e77a5c7bd1fb4e23bfd9f01',
  '5e77a5c7bd1fb4e23bfd9ef4',
  '5e77a5c7bd1fb4e23bfd9edf'],
 'foodmod_list': {},
 'order_type': 'user'}

In [80]:
Table.objects[2].to_dbref()

DBRef('table', ObjectId('5e77a5c9bd1fb4e23bfd9f0a'))

In [86]:
input_order={'placedby' : ["5e77a5d0bd1fb4e23bfd9f16"],'table' : '5e77a5c9bd1fb4e23bfd9f0a','foodlist' : ['5e77a5c7bd1fb4e23bfd9f01','5e77a5c7bd1fb4e23bfd9ed0'],'foodmod_list':{},'order_type':'table'}
input_order

{'placedby': ['5e77a5d0bd1fb4e23bfd9f16'],
 'table': '5e77a5c9bd1fb4e23bfd9f0a',
 'foodlist': ['5e77a5c7bd1fb4e23bfd9f01', '5e77a5c7bd1fb4e23bfd9ed0'],
 'foodmod_list': {},
 'order_type': 'table'}

In [89]:
order_placement(input_order)

In [88]:
def order_placement(input_order):
    users_list = []
    for userid in input_order['placedby']:
        users_list.append(User.objects.get(id=userid).to_dbref())
    ordered_list = []
    for foodid in input_order['foodlist']:
        ordered_list.append(FoodItemMod.from_json(FoodItem.objects(id=foodid).exclude("id")[0].to_json()))
    ordered_table = Table.objects.get(id=input_order['table'])
    order_ob=Order(placedby = users_list,table = ordered_table.to_dbref(),foodlist={str(key):ordered_list[key] for key in range(len(ordered_list))},customer_type=input_order['order_type']).save()
    ordered_table.update(push__orders=order_ob.to_dbref())

In [35]:
users_list = []
for userid in input_order['placedby']:
    users_list.append(User.objects.get(id=userid).to_dbref())
    

ordered_list = []
for foodid in input_order['foodlist']:
    ordered_list.append(FoodItemMod.from_json(FoodItem.objects(id=input_order['foodlist'][2]).exclude("id")[0].to_json()))
    

In [54]:
Order(placedby = users_list,table = Table.objects.get(id=str(table_list[0].id)).to_dbref(),foodlist={str(key):ordered_list[key] for key in range(len(ordered_list))},customer_type=input_order['order_type']).save()

<Order: Order object>

In [90]:
Order.objects[3].to_json()

'{"_id": {"$oid": "5e786083bd1fb4e23bfd9f1c"}, "placedby": [{"$oid": "5e77a5d0bd1fb4e23bfd9f16"}], "table": {"$oid": "5e77a5c9bd1fb4e23bfd9f0a"}, "foodlist": {"0": {"name": "THAI GRILLED FISH IN BANANA LEAF", "description": "Delicately Grilled Fish With Basil Flavour And Served Hot", "price": "330"}, "1": {"name": "CRISPY CHILLY MUSHROOM", "description": "Crispy Fried Mushrooms Tossed With Oriental Spices", "price": "220"}}, "customer_type": "table"}'

In [169]:
classes_json=json.loads(Restaurant.objects[0].to_json())

In [170]:
with open('json_with_class.json','w') as f:
    f.write(str(classes_json))

# Pages

In [14]:
pages={}

In [15]:
pages[0]="""$BAR BITES

NACHOS (Veg/Chicken)

Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 

190/230

CHEESE MASALA PAPAD

Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese

160

H.O.C SIGNATURE MASALA PEANUTS

A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 

160

SPICY TRICOLOUR CHAAT

Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions

180

CHATPATA CHANA CHAAT

A Delectable Blend Of Bengal Gram And A Tangy Mixture

160

TANDOORI CHICKEN CHAAT

Clay Oven Roasted Chicken Blended With Classic Spices

220

CHEESE DYNABITES

Classic Cheese Balls Served with New Orleans Dip

220

$SALADS

GREEN SALAD

Traditional Green Cuts With Mint Yoghurt Dip

170

HOC SALAD (Veg/Chicken)

Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing

180/210

WATERMELON & FETA

An All Time Classic - With Balsamic Glaze

210

CAESAR SALAD (Veg/Chicken)

Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing

190/230"""

In [16]:
pages[1]="""$VEG STARTERS

CHEESE & CORN TRIANGLE (new)

Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried

220

CHAMPIGNONS FARCIS (new)

Sauteed Button Mushroom Stuffed With Cheese & Herbs

230

CHILLI CHEESE CIGARA (new)

Deep Fried Rolls With Cheese, Chill & Herbs

220

HOC CHIP & DIP BASKET

Combination Of Onion Rings, Jalapeno Cheese Balls, Greek Potato Wedges, Habanero Green Chili Fries

390

FRENCH FRIES

French Fries Tossed With Hoc Signature Cajun Spice Seasoning

190

ONION RINGS

Extra Crispy Onion Rings With A Spicy Salsa

190

MUSHROOM LOADED FRIES

Cheesy French Fries Topped With Mushrooms And Herbs

230

GARLIC BREAD

Garlic Butter Flavoured Bread Topped With Cheese

190

CHEESE CORN TIKKI

American Com & Cottage Cheese Patty Served with Mint Chutney

220

CRISPY CHILLY MUSHROOM

Crispy Fried Mushrooms Tossed With Oriental Spices

220

CRISPY CHILLY BABY CORN

Crispy Fried Babycom Tossed With Oriental Spices

190

CRISPY CORN CHILLY PEPPER

Fred Undi Lightly Crispy & Seasoned With Lime Juice, Salt, Chilli & Pepper

240

HONEY CHILLY POTATO

Crispy Fried Potatoes Tossed With Chilli And Honey

190"""

In [17]:
pages[2]="""$VEG STARTERS

SESAME CHILLI PANEER

Cottage Cheese Cubes Tossed In A Sweet & Spicy Sauce, Finished With Sesame Seeds

270

PANDAN LEAF COTTAGE CHEESE

Tender Cottage Cheese Wrapped In Fragrant Pandan Leaves, With Chilli Coriander Sauce

280

MEZZE BASKET

A Combination Of Hummus, Mutabal & Labneh, Served With Pita Bread

340

$FROM THE CLAY OVEN

MAKHMALI PANEER TIKKA

Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices

270

ANGARI PANEER TIKKA

Clay Oven Roasted Spicy Morsels Of Cottage Cheese

270

TANDOORI STUFFED MUSHROOMS

Mushrooms Filled With A Blend Of Indian Spices And Tangy Stuffing Mix, Adding A Unique Taste On The Bite

250

TANDOORI VEG MOMOS

Soft And Juicy Vegetable Momos Marinated With Tandoori Masala And Char Grilled

220

$CHICKEN STARTERS

CHICKEN LOADED FRIES

Cheesy French Fries Topped With Chicken & Herbs

260

SHANG DONG CHICKEN LOLLY POP

Crispy Lolly pops Crunched To Perfection & Tossed in our Signature Shang Dong Drip

260"""

In [18]:
pages[3]="""$CHICKEN STARTERS

HOC SIGNATURE CHICKEN WINGS

Sweet Chilly / Spiced Szechuan

260

PEPPER GARLIC CHICKEN

Cubes Of Chicken Tossed With Garlic And Pepper In Oriental Style

260

GUNTUR JUNCTION CHICKEN

Chicken Cubes Tossed With Freshly Ground Red Spices And Finished In South Indian Style

270

ANDHRA CHILLI CHICKEN *VERY HOT*

All Time Favourite Andhra Style Chicken Cooked With Coriander And Green Chilli Paste And Fresh Green Chillies

270

LEMON PEPPER CHICKEN

Slice Chicken Wok Tossed, Finished With Pepper Garlic & Lime

260

JIANG CHILLI CHICKEN

Chicken Tossed With Shredded Ginger And Chilli In Jiang Style

260

CHICKEN GHEE ROAST

Chicken Roasted With A Fiery Red, Tangy And Spicy With The Flavour Of Ghee

280

MASALA FRY CHICKEN (new)

Classic Indian Style With Bone Deep fried Chicken Served With Mint Chutney

250

$FROM THE CLAY OVEN

JAIPURI MURGH KA SOOLA

Succulent Pieces Of Chicken Zapped With Red Spices, Cinnamon Flavour & Finished In Tandoor

270

TANDOORI MURGH (HALF/FULL)

Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor

250/480

TANDOORI POMFRET

Indian Spices Marinated Whole Pomfret Skewered And Finished In Clay Oven

590"""

In [19]:
pages[4]="""$FROM THE CLAY OVEN

FISH TIKKA KALIMIRCH

Fresh Cubes Of Basa Fish Marinated With Crushed Black Pepper & Indian Spices, Finished In Clay Oven

320

AMRITSAR FISH TIKKA

Spicy Fish Tikka Marinated In Traditional Spices And Char Grilled

320

KALMI KEBAB (3pcs)

Clay Oven Roasted Chicken Drumsticks

260

ANGARI MURGH TIKKA

Clay Oven Roasted Spicy Morsels Of Chicken

260

TANDOORI LOLLIPOPS

Chicken Lollipops Marinated With Aromatic Indian Spices And Cooked In The Tandoor

270

MURGH MALAI TIKKA

Clay Oven Roasted Creamy Chunks Of Chicken

260

MURGH KASTURI KEBAB

Boneless Charcoal Smoked Chicken Tikka With Kasturi Methi Flavoured

260

TANDOORI CHICKEN MOMOS

Soft And Juicy Chicken Momos Marinated With Tandoori Masala And Char Grilled

260

TANDOORI KING PRAWNS

King Prawns Marinated With Yogurt, Lime Juice, Bhatti Spices And Slowly Cooked In Tandoor

499

$LAMB STARTERS

ANDHRA LAMB PEPPER FRY

Tender Mutton Pieces Tossed In South Indian Spices

350

MUTTON GHEE ROAST

Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee

370"""

In [20]:
pages[5]="""$LAMB STARTERS

LAHORI SEEKH KEBAB

Succulent Minced Lamb Kebabs In The True Lahori Street Food Style

370

RAILWAY MUTTON DRY

With Bone Mutton Cooked In Rich Indian Spices, As Served In The Railway Canteen Of The British Raj

350

$EGG STARTERS

EGG BHURJI

Desi Style Scrambled Eggs

170

SPICY MASALA OMELETTE

Slow Cooked Egg Omlet With Masala

180

EGG CHILLI 

Spicy Boiled Egg With Chilli Fry

210

$FROM THE GRILL

PERI PERI PANEER/CHICKEN

Peri Peri Basted Paneer / Chicken Cubes Served With Tzatziki Dip

260/280

THAI GRILLED FISH IN BANANA LEAF

Delicately Grilled Fish With Basil Flavour And Served Hot

330

HARISSA GRILLED FISH

Fish Marinated In A Spicy Tunisian Harissa Sauce And Grilled To Perfection

330

THAI GRILLED CHICKEN

Boneless Chicken Marinated With Thai Spices And Grilled

270"""

# Consolidating

## Pages to DF

In [21]:
half_the_menu=[]

for i in range(6):
    half_the_menu.append(pages[i])

half_menu_text="\n\n".join(half_the_menu)

In [22]:
str_menu = pd.DataFrame(columns=['sub_category','name','description','price'])

In [23]:
description=dish_name=price=subcat=''
counter=0
for line in half_menu_text.split("\n\n"):
    if(re.search("^[\$]",line)):
        subcat = line[1:].strip()
    else:
        if(counter%3==0):
            dish_name = line.strip()
        elif(counter%3==1):
            description = line.strip()
        elif(counter%3==2):
            price = line.strip()
            str_menu.loc[str_menu.shape[0]]=[subcat,dish_name,description,price]
        counter+=1

## creating main categories

In [24]:
main_category={"BAR BITES":"BAR BITES","SALADS":"SALADS","VEG STARTERS":"VEG STARTERS","CHICKEN STARTERS":"NON-VEG STARTERS","LAMB STARTERS":"NON-VEG STARTERS","EGG STARTERS":"NON-VEG STARTERS","FROM THE GRILL":"NON-VEG STARTERS","FROM THE CLAY OVEN":"CHANGE"}

In [25]:
str_menu.loc[:,"main_category"]=str_menu['sub_category'].apply(lambda x: main_category[x])

In [26]:
str_menu.loc[27,"main_category"]="VEG STARTERS"

str_menu.loc[28,"main_category"]="VEG STARTERS"

str_menu.loc[29,"main_category"]="VEG STARTERS"

str_menu.loc[30,"main_category"]="VEG STARTERS"

In [27]:
str_menu.loc[str_menu['main_category']=='CHANGE',"main_category"]="NON-VEG STARTERS"

In [28]:
str_menu.columns

Index(['sub_category', 'name', 'description', 'price', 'main_category'], dtype='object')

In [30]:
str_menu=str_menu[['main_category','sub_category', 'name', 'description', 'price']]

## Taking care of multiple prices

In [44]:
lengths = str_menu['price'].str.split('/').map(len)

In [59]:
str_menu = pd.DataFrame({'main_category': np.repeat(str_menu['main_category'], lengths),
                    'sub_category': np.repeat(str_menu['sub_category'], lengths),
                    'name': np.repeat(str_menu['name'],lengths),
                    'description': np.repeat(str_menu['description'],lengths),
                    'price': chainer(str_menu['price'])})

In [81]:
str_menu.reset_index(inplace=True,drop=True)

In [165]:
str_menu.loc[45,'price']="250"

In [83]:
pickle.dump(str_menu,open('half_menu_main.pkl','wb'))

In [32]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

## COnverting into json

In [33]:
final_list_json=[]

In [35]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

In [4]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

In [9]:
from pprint import pprint

In [1]:
json.load(Restaurant.objects[0].to_json())

NameError: name 'Restaurant' is not defined

# js.data

In [17]:
pickle.dump(final_json,open('jason.pkl','wb'))

In [11]:
pickle.dump(final_list_json,open('jason.pkl','wb'))